In [1]:
import pandas as pd

In [2]:
import numpy as np

In [4]:
from dynamic_portfolio.utils import load_csv
from dynamic_portfolio.utils import features_creation
from dynamic_portfolio.preprocess import scaler

In [6]:
df = features_creation('AAPL')

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [7]:
pd.set_option('display.max_columns', None)

In [8]:
df

,date,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,volume_momentum_5days,volume_momentum_10days,volume_momentum_20days,price/eps,momentum_5days/eps,momentum_10days/eps,momentum_20days/eps,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,2001-01-18,21910900.0,0.0,1.0,-0.0097,0.0,0.0512,0.0472,0.0040,30.42,167108.0,0.0593,0.042,0.03,130668.0,175.100,226791.0,46519.0,264.8000,109.940002,0.111838,0.064132,0.052955,0.052157,0.040465,-0.003374,0.013462,0.009728,-0.014733,-0.031569,-0.027836,0.702813,0.700027,0.580735,10553160.0,10511320.0,8720080.00,-29.325744,0.347883,-1.387804,-1.002894,0.005124,-0.013487,-0.024793,0.142857,0.021834,-0.007851,0.0,0.006322,-0.021521,0.011656
1,2001-01-19,13869000.0,0.0,1.0,-0.0097,0.0,0.0519,0.0475,0.0044,32.12,167108.0,0.0592,0.042,0.03,130668.0,175.100,226791.0,46519.0,264.6000,110.660004,0.043339,0.046549,0.068873,0.054401,0.046476,0.026323,0.014632,0.014644,0.085515,0.097206,0.097194,0.586974,0.513140,0.434703,12861120.0,11243350.0,9524725.00,-30.596683,-2.713713,-1.508461,-1.509722,-0.000755,0.013672,0.006356,0.100000,0.055884,0.006549,0.0,0.006322,-0.021521,0.011656
2,2001-01-22,9273700.0,0.0,1.0,-0.0097,0.0,0.0525,0.0475,0.0050,32.21,167108.0,0.0600,0.042,0.03,130668.0,175.100,226791.0,46519.0,266.7000,110.239998,-0.012821,0.063991,0.061658,0.054469,0.046250,0.017599,0.014751,0.017688,0.025739,0.028588,0.025650,0.920358,0.815481,0.712686,12764440.0,11309900.0,9884240.00,-30.204418,-1.814377,-1.520713,-1.823549,0.007937,0.011561,0.000000,0.136364,0.002802,-0.003795,0.0,0.006322,-0.021521,0.011656
3,2001-01-23,15705900.0,0.0,1.0,-0.0097,0.0,0.0530,0.0482,0.0048,31.66,167108.0,0.0597,0.042,0.03,130668.0,175.100,226791.0,46519.0,266.3000,110.239998,0.064935,0.098636,0.054787,0.052003,0.046681,0.024035,0.017513,0.015690,-0.036856,-0.030334,-0.028511,1.413365,1.240165,1.061406,13107120.0,11500920.0,9843165.00,-32.165744,-2.477871,-1.805507,-1.617553,-0.001500,0.009524,0.014737,-0.040000,-0.017075,0.000000,0.0,0.006322,-0.021521,0.011656
4,2001-01-24,12805200.0,0.0,1.0,-0.0097,0.0,0.0533,0.0481,0.0052,31.46,167108.0,0.0605,0.042,0.03,130668.0,175.100,226791.0,46519.0,264.7000,111.680000,0.000000,0.057771,0.054628,0.054026,0.047021,0.037837,0.022846,0.020050,0.027098,0.042089,0.044885,0.964925,0.789779,0.655863,15155020.0,12404190.0,10300920.00,-32.165744,-3.900700,-2.355285,-2.066976,-0.006008,0.005660,-0.002075,0.083333,-0.006317,0.013062,0.0,0.006322,-0.021521,0.011656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,2022-11-16,64218266.0,0.0,1.0,1.2900,0.0,0.0367,0.0435,-0.0068,85.62,273451.0,0.0383,0.037,0.05,154369.0,298.012,597492.0,60082.0,1773.7700,106.279999,-0.008331,0.017516,0.044560,0.037662,0.033453,0.016286,-0.001387,0.003215,-0.004416,0.013256,0.008655,0.994617,1.049329,1.038195,89384549.0,94301465.3,93300813.65,115.341085,0.012625,-0.001075,0.002492,-0.002519,-0.034211,-0.004577,0.192982,-0.014389,-0.001128,0.0,0.000000,0.000000,-0.001197
5430,2022-11-17,80389400.0,0.0,1.0,1.2900,0.0,0.0377,0.0443,-0.0066,81.69,273451.0,0.0383,0.037,0.05,154369.0,298.012,597492.0,60082.0,1760.8700,106.690002,0.012971,0.036469,0.045745,0.037334,0.033515,0.013785,-0.000465,0.002328,-0.022116,-0.007866,-0.010659,1.311878,1.443287,1.425683,84246506.8,92685357.4,91554896.45,116.837209,0.010686,-0.000361,0.001804,-0.007273,0.027248,0.018391,-0.029412,-0.045900,0.003858,0.0,0.0000

In [10]:
final_df = scaler(df)

In [11]:
final_df

,date,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,volume_momentum_5days,volume_momentum_10days,volume_momentum_20days,price/eps,momentum_5days/eps,momentum_10days/eps,momentum_20days/eps,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,2001-01-18,-0.346003,0.0,0.0,-0.520563,0.0,1.683449,2.008561,-1.025447,-1.302865,-1.106639,2.815027,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620106,1.792221,0.111838,1.983617,2.728459,2.969448,2.045750,-0.491008,1.738754,1.683579,-0.725595,-1.461834,-1.257414,-1.224829,-1.146544,-1.412197,-0.772641,-0.792025,-0.876532,-0.986475,0.062426,-0.619727,-0.682886,0.433807,-0.548798,-0.525452,0.685932,0.415832,-1.588763,0.0,0.661811,-2.436691,-0.129723
1,2001-01-19,-0.614623,0.0,0.0,-0.520563,0.0,1.741110,2.029087,-0.979457,-1.237085,-1.106639,2.808797,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620504,1.858971,0.043339,1.038557,4.023115,3.177779,2.666917,2.544799,1.907877,2.682564,4.210961,4.500903,4.393001,-1.610149,-1.688337,-1.797802,-0.690418,-0.765302,-0.846398,-1.000526,-0.844069,-0.669428,-0.995584,-0.106335,0.535720,0.104186,0.479318,1.062331,1.322936,0.0,0.661811,-2.436691,-0.129723
2,2001-01-22,-0.768118,0.0,0.0,-0.520563,0.0,1.790534,2.029087,-0.910473,-1.233602,-1.106639,2.858633,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.616334,1.820033,-0.012821,1.976073,3.436255,3.184096,2.643568,1.653038,1.925050,3.301135,1.267393,1.323638,1.159753,-0.501205,-0.811839,-1.063774,-0.693862,-0.762872,-0.832934,-0.996189,-0.577788,-0.674475,-1.189207,0.692156,0.451417,-0.024290,0.654627,0.054477,-0.768696,0.0,0.661811,-2.436691,-0.129723
3,2001-01-23,-0.553266,0.0,0.0,-0.520563,0.0,1.831721,2.076982,-0.933468,-1.254884,-1.106639,2.839945,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.617128,1.820033,0.064935,3.838197,2.877404,2.955185,2.688142,2.310944,2.324240,2.895104,-1.814998,-1.404638,-1.287929,1.138698,0.419339,-0.142960,-0.681654,-0.755899,-0.834472,-1.017874,-0.774240,-0.791786,-1.062113,-0.174732,0.370081,0.273594,-0.195624,-0.322929,-0.001263,0.0,0.661811,-2.436691,-0.129723
4,2001-01-24,-0.650157,0.0,0.0,-0.520563,0.0,1.856433,2.070140,-0.887478,-1.262623,-1.106639,2.889780,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620305,1.953532,0.000000,1.641733,2.864462,3.142905,2.723276,3.721787,3.094854,3.780943,1.334317,1.948782,2.029040,-0.352958,-0.886348,-1.213817,-0.608695,-0.722924,-0.817330,-1.017874,-1.195519,-1.018250,-1.339395,-0.588911,0.215807,-0.066227,0.398967,-0.118665,2.639928,0.0,0.661811,-2.436691,-0.129723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,2022-11-16,1.067172,0.0,0.0,1.920188,0.0,0.489039,1.755402,-2.267170,0.833050,1.298006,1.506848,-0.739130,3.333333,1.418862,1.659773,2.233977,1.796868,1.376334,1.452910,-0.008331,-0.521921,2.045629,1.623760,1.321049,1.518759,-0.406861,0.360028,-0.217573,0.613723,0.391685,-0.254195,-0.133902,-0.204252,2.035810,2.266810,2.290988,0.612943,-0.036839,-0.048508,-0.062591,-0.268393,-1.376304,-0.116800,0.927587,-0.271928,-0.229311,0.0,-0.426518,-0.373957,-0.999994
5430,2022-11-17,1.607330,0.0,0.0,1.920188,0.0,0.571412,1.810139,-2.244175,0.680982,1.298006,1.506848,-0.739130,3.333333,1.418862,1.659773,2.233977,1.796868,1.350718,1.490921,0.012971,0.496786,2.142008,1.593324,1.327480,1.263072,-0.273751,0.179782,-1.089147,-0.364285,-0

In [34]:
X = final_df.drop(columns=['date'])

In [35]:
X

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,volume_momentum_5days,volume_momentum_10days,volume_momentum_20days,price/eps,momentum_5days/eps,momentum_10days/eps,momentum_20days/eps,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,-0.346003,0.0,0.0,-0.520563,0.0,1.683449,2.008561,-1.025447,-1.302865,-1.106639,2.815027,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620106,1.792221,0.111838,1.983617,2.728459,2.969448,2.045750,-0.491008,1.738754,1.683579,-0.725595,-1.461834,-1.257414,-1.224829,-1.146544,-1.412197,-0.772641,-0.792025,-0.876532,-0.986475,0.062426,-0.619727,-0.682886,0.433807,-0.548798,-0.525452,0.685932,0.415832,-1.588763,0.0,0.661811,-2.436691,-0.129723
1,-0.614623,0.0,0.0,-0.520563,0.0,1.741110,2.029087,-0.979457,-1.237085,-1.106639,2.808797,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620504,1.858971,0.043339,1.038557,4.023115,3.177779,2.666917,2.544799,1.907877,2.682564,4.210961,4.500903,4.393001,-1.610149,-1.688337,-1.797802,-0.690418,-0.765302,-0.846398,-1.000526,-0.844069,-0.669428,-0.995584,-0.106335,0.535720,0.104186,0.479318,1.062331,1.322936,0.0,0.661811,-2.436691,-0.129723
2,-0.768118,0.0,0.0,-0.520563,0.0,1.790534,2.029087,-0.910473,-1.233602,-1.106639,2.858633,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.616334,1.820033,-0.012821,1.976073,3.436255,3.184096,2.643568,1.653038,1.925050,3.301135,1.267393,1.323638,1.159753,-0.501205,-0.811839,-1.063774,-0.693862,-0.762872,-0.832934,-0.996189,-0.577788,-0.674475,-1.189207,0.692156,0.451417,-0.024290,0.654627,0.054477,-0.768696,0.0,0.661811,-2.436691,-0.129723
3,-0.553266,0.0,0.0,-0.520563,0.0,1.831721,2.076982,-0.933468,-1.254884,-1.106639,2.839945,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.617128,1.820033,0.064935,3.838197,2.877404,2.955185,2.688142,2.310944,2.324240,2.895104,-1.814998,-1.404638,-1.287929,1.138698,0.419339,-0.142960,-0.681654,-0.755899,-0.834472,-1.017874,-0.774240,-0.791786,-1.062113,-0.174732,0.370081,0.273594,-0.195624,-0.322929,-0.001263,0.0,0.661811,-2.436691,-0.129723
4,-0.650157,0.0,0.0,-0.520563,0.0,1.856433,2.070140,-0.887478,-1.262623,-1.106639,2.889780,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620305,1.953532,0.000000,1.641733,2.864462,3.142905,2.723276,3.721787,3.094854,3.780943,1.334317,1.948782,2.029040,-0.352958,-0.886348,-1.213817,-0.608695,-0.722924,-0.817330,-1.017874,-1.195519,-1.018250,-1.339395,-0.588911,0.215807,-0.066227,0.398967,-0.118665,2.639928,0.0,0.661811,-2.436691,-0.129723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,1.067172,0.0,0.0,1.920188,0.0,0.489039,1.755402,-2.267170,0.833050,1.298006,1.506848,-0.739130,3.333333,1.418862,1.659773,2.233977,1.796868,1.376334,1.452910,-0.008331,-0.521921,2.045629,1.623760,1.321049,1.518759,-0.406861,0.360028,-0.217573,0.613723,0.391685,-0.254195,-0.133902,-0.204252,2.035810,2.266810,2.290988,0.612943,-0.036839,-0.048508,-0.062591,-0.268393,-1.376304,-0.116800,0.927587,-0.271928,-0.229311,0.0,-0.426518,-0.373957,-0.999994
5430,1.607330,0.0,0.0,1.920188,0.0,0.571412,1.810139,-2.244175,0.680982,1.298006,1.506848,-0.739130,3.333333,1.418862,1.659773,2.233977,1.796868,1.350718,1.490921,0.012971,0.496786,2.142008,1.593324,1.327480,1.263072,-0.273751,0.179782,-1.089147,-0.364285,-0.481144,0.801119,1.008195,0.818930,1.852762,2.207812,2.225604,0.629484,-0.037413,-0.04

In [36]:
y = final_df['return']

In [17]:
from sklearn.model_selection import TimeSeriesSplit

In [38]:
ts_cv = TimeSeriesSplit(
    n_splits=90,
    gap=0,
    max_train_size=252,
    test_size=45,
)

In [39]:
all_splits = list(ts_cv.split(X, y))

In [28]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

In [26]:
model = HistGradientBoostingRegressor()

In [48]:
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.6f} +/- {mae.std():.6f}\n"
        f"Root Mean Squared Error: {rmse.mean():.6f} +/- {rmse.std():.6f}"
    )


In [44]:
import seaborn as sns

In [49]:
evaluate(model, X, y, ts_cv)

Mean Absolute Error:     0.002954 +/- 0.002445
Root Mean Squared Error: 0.005486 +/- 0.004490


In [46]:
scores = cross_val_score(model, X, y, cv = ts_cv)

In [47]:
scores = np.mean(scores)
scores

0.9102020852625771